# Data Engineering tasks
This file containes the descriptions of the solutions for all the 3 tasks.

## SparkSession initialization
Defining the entry point into all functionality in Spark by creating a `SparkSession` class. 

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
            .appName("Data Engineering Test")
            .master("local[*]")
            .getOrCreate()

spark = org.apache.spark.sql.SparkSession@3dafd0f


Spark local-1575207444697: Some(http://dusan:4042)

## 1. Sales and rentals broadcast rights
In task 1 the two datasets, i.e. `whatson` and `started_streams` are joined based on two joint conditions. The first condition implies that the variable `house_number` should be the identical in both datasets. The other condition requires the countries to be the same. Then, the requested columns are selected from the joined dataset and the same dataset is filtered making the date from the first dataset(`started_streams`) being in the broadcast_right range of whatson data. 

### 1.1 DataFrame and extraction of requested data
Two `DataFrame`'s are created based on the content of `whatson.csv` and `started_streams.csv` files, and named `whatson_df` and `streams_df` respectively.
To be able to join them based on the `house_number` and `country_code`(`broadcast_right_region` for `whatson_df`) the values in the data has to be comparable (e.g. se should map to Sweden).
In order to achieve that, `countryCodeMap` maps a country to its corresponding code to make it possible to compare the countries of the two datasets.
The dataframes are filtered based on the `broadcast_right_vod_type` for `whatson_df` and `product_type` for `streams_df` so that only the data with `svod` or `tvod` is kept.

In [16]:
import org.apache.spark.sql.functions.lower
import org.apache.spark.sql.functions.typedLit
import org.apache.spark.sql.Column

val countryCodeMap: Column = typedLit(Map(
  "Norway" -> "no",
  "Sweden"-> "se",
  "Denmark" -> "dk",
  "Finland" -> "fi"
))

val whatson_df = spark.read.format("csv")
                .option("header", "true")
                .load("data/whatson.csv")
                .withColumn("broadcast_right_vod_type",lower($"broadcast_right_vod_type"))
                .filter($"broadcast_right_vod_type" === "tvod" || $"broadcast_right_vod_type" === "svod")
                .withColumn("broadcast_right_region", countryCodeMap($"broadcast_right_region"))

val streams_df = spark.read.format("csv")
                .option("delimiter",";")
                .option("header", "true")
                .load("data/started_streams.csv")
                .filter($"product_type" === "tvod" || $"product_type" === "svod")
                
//whatson_df.select("broadcast_right_region").distinct().show()
//streams_df.select("country_code").distinct().show()

//println(whatson_df.select("title").distinct().count())
//println(streams_df.select("program_title").distinct().count())

//whatson_df.select("product_category").distinct().show()
//streams_df.select("genre").distinct().show()

//streams_df.select("dt").distinct().show()
//whatson_df.select("dt").distinct().show()

println(streams_df.count()) // |99 829|
println(whatson_df.count()) // |875 224|

//streams_df.printSchema()
//whatson_df.printSchema()

//whatson_df.select("broadcast_right_vod_type").distinct().show()
//streams_df.select("product_type").distinct().show()


99829
875224


countryCodeMap = keys: [Norway,Sweden,Denmark,Finland], values: [no,se,dk,fi]
whatson_df = [dt: string, house_number: string ... 6 more fields]
streams_df = [dt: string, time: string ... 9 more fields]


lastException: Throwable = null


[dt: string, time: string ... 9 more fields]

### 1.2 Join 
The inner join is performed based on a `house_number` and country.
After that, the majority of respective columns from `streams_df` are selected and only keeping `broadcase_right` columns from `whatson`, given the desired outcome of the task.  
For the simplicity reasons, `whatson_df` and `streams_df` got aliases `df1` and `df2` respectively.

In [17]:
val joined_df = whatson_df.as("df1").join(streams_df.as("df2"),
                               $"df1.house_number"===$"df2.house_number"&&
                               $"df1.broadcast_right_region"===$"df2.country_code", "inner")
                            .select("df2.dt",
                                    "df2.time",
                                    "df2.device_name",
                                    "df2.house_number",
                                    "df2.user_id",
                                    "df2.country_code",
                                    "df2.program_title",
                                    "df2.season",
                                    "df2.season_episode",
                                    "df2.genre",
                                    "df2.product_type",
                                    "df1.broadcast_right_start_date",
                                    "df1.broadcast_right_end_date")
                            

println(joined_df.count()) // |1 571 021|


1571021


joined_df = [dt: string, time: string ... 11 more fields]


[dt: string, time: string ... 11 more fields]

### 1.3 Filtering out old dates 
The final dataframe is obtained by filtering out the `dt`(dates from `started_streams`) that are outiside of range `broadcast_right_start_date` and `broadcast_right_end_date`. 
Lastly, the results are saved localy.

In [21]:
import org.apache.spark.sql.functions._
val final_broadcast_df = joined_df.filter($"dt".between($"broadcast_right_start_date",
                                                        $"broadcast_right_end_date"))

//final_broadcast_df.agg(min("dt")).show() |2018-10-01|
//final_broadcast_df.agg(max("dt")).show() |2018-10-01|

//final_broadcast_df.agg(min("broadcast_right_start_date"),        |1900-01-01|
//                       max("broadcast_right_start_date")).show() |2018-10-01|

//final_broadcast_df.agg(min("broadcast_right_end_date"),          |2018-10-01|
//                       max("broadcast_right_end_date")).show()   |2100-12-31|

final_broadcast_df.write
                  .csv("task1/output")
//println(final_broadcast_df.count()) // |931 268|


931268
root
 |-- dt: string (nullable = true)
 |-- time: string (nullable = true)
 |-- device_name: string (nullable = true)
 |-- house_number: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- program_title: string (nullable = true)
 |-- season: string (nullable = true)
 |-- season_episode: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- product_type: string (nullable = true)
 |-- broadcast_right_start_date: string (nullable = true)
 |-- broadcast_right_end_date: string (nullable = true)



final_broadcast_df = [dt: string, time: string ... 11 more fields]


[dt: string, time: string ... 11 more fields]

## 2. Product and user count

In [67]:
import org.apache.spark.sql.functions._

val streams_df = spark.read.format("csv")
                .option("delimiter",";")
                .option("header", "true")
                .load("data/started_streams.csv")

val watches_count = streams_df.groupBy("program_title")
                              .count()
                              .withColumnRenamed("count", "content_count")

val unique_users = streams_df.groupBy("program_title")
                             .agg('program_title, countDistinct('user_id).as("unique_users"))

val streams_counts_df =  streams_df.join(watches_count, Seq("program_title"))
val product_user_df =  streams_counts_df.join(unique_users,Seq("program_title"))
                                        .select("dt",
                                                "program_title",
                                                "device_name",
                                                "country_code",
                                                "product_type",
                                                "unique_users",
                                                "content_count")

product_user_df.write
                  .csv("task2/output")


// println(streams_df.count()) |100 000|
// println(product_user_df.count()) |100 000|

100000


streams_df = [dt: string, time: string ... 9 more fields]
watches_count = [program_title: string, content_count: bigint]
unique_users = [program_title: string, program_title: string ... 1 more field]
streams_counts_df = [program_title: string, dt: string ... 10 more fields]
product_user_df = [dt: string, program_title: string ... 5 more fields]


[dt: string, program_title: string ... 5 more fields]

## 3. Genre and time of day